# Bayesian Mixture Model

In [9]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.append('../')

import beer
import numpy as np

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def plot_normal(fig, mean, cov, n_sigma=2, alpha=1., color='blue'):
    'Plot a Normal density'
    # Eigenvalue decomposition of the covariance matrix.
    evals, evecs = np.linalg.eigh(cov)
    
    sign = 1 if cov[1, 0] == 0 else np.sign(cov[1, 0])
    # Angle of the rotation.
    angle =  - np.arccos(sign * abs(evecs[0, 0]))
   
    fig.ellipse(x=mean[0], y=mean[1], 
                width=n_sigma * 2* np.sqrt(evals[0]), 
                height=n_sigma * 2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)
    
    fig.ellipse(x=mean[0], y=mean[1], 
                width=2 * np.sqrt(evals[0]), 
                height=2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)
    
def plot_mixture(fig, gmm, n_sigma=2, alpha=1., color='blue'):
    weights = gmm.weights
    for i, component in enumerate(gmm.components):
        plot_normal(fig, component.mean, component.cov, n_sigma, alpha * weights[i], color)

Generate some normally distributed data:

In [14]:
fig = figure(
    title='Data',
    x_range=(-10, 10),
    y_range=(-10, 10),
    width=400,
    height=400,
)

mean = np.array([-1.5, 4]) 
cov = np.array([
    [.75, 0],
    [0, 2.]
])
data1 = np.random.multivariate_normal(mean, cov, size=500)

plot_normal(fig, mean, cov, alpha=0.1)

mean = np.array([5, 5]) 
cov = np.array([
    [2, 1],
    [1, .75]
])
data2 = np.random.multivariate_normal(mean, cov, size=500)

plot_normal(fig, mean, cov, alpha=0.1)

data = np.vstack([data1, data2])
print(data.shape)
#np.random.shuffle(data)

fig.circle(data[:, 0], data[:, 1], alpha=.1)

show(fig)

(1000, 2)


In [6]:
p_mean = data.mean(axis=0)
p_cov = np.cov(data.T)
args = {'dim':2, 'mean':p_mean, 'cov': p_cov, 'prior_count':1e-6, 'random_init':True}
gmm_diag = beer.Mixture.create(10, beer.NormalDiagonalCovariance.create, args, prior_count=1e-6)
gmm_full = beer.Mixture.create(10, beer.NormalFullCovariance.create, args, prior_count=1e-6)
gmm_diag.fit(data, max_epochs=100)
gmm_full.fit(data, max_epochs=100, verbose=True)

ln p(X) = -2,900,026.374 E[ln p(X|...)] = -2,900,026.374 D(q || p) = 0.000
ln p(X) = -2.113       E[ln p(X|...)] = -1.665       D(q || p) = 0.448
ln p(X) = -2.053       E[ln p(X|...)] = -1.654       D(q || p) = 0.399
ln p(X) = -2.007       E[ln p(X|...)] = -1.650       D(q || p) = 0.357
ln p(X) = -1.964       E[ln p(X|...)] = -1.647       D(q || p) = 0.317
ln p(X) = -1.934       E[ln p(X|...)] = -1.646       D(q || p) = 0.288
ln p(X) = -1.914       E[ln p(X|...)] = -1.645       D(q || p) = 0.269
ln p(X) = -1.872       E[ln p(X|...)] = -1.643       D(q || p) = 0.228
ln p(X) = -1.845       E[ln p(X|...)] = -1.643       D(q || p) = 0.201
ln p(X) = -1.844       E[ln p(X|...)] = -1.643       D(q || p) = 0.201
ln p(X) = -1.844       E[ln p(X|...)] = -1.642       D(q || p) = 0.201
ln p(X) = -1.843       E[ln p(X|...)] = -1.642       D(q || p) = 0.201
ln p(X) = -1.842       E[ln p(X|...)] = -1.642       D(q || p) = 0.201
ln p(X) = -1.842       E[ln p(X|...)] = -1.641       D(q || p) = 0.200
ln

In [16]:
fig1 = figure(
    title='GMM (diag)',
    x_range=(-10, 10),
    y_range=(-10, 10),
    width=400,
    height=400,
)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plot_mixture(fig1, gmm_diag, color='red')

fig2 = figure(
    title='GMM (full)',
    x_range=(-10, 10),
    y_range=(-10, 10),
    width=400,
    height=400,
)
fig2.circle(data[:, 0], data[:, 1], alpha=.1)
plot_mixture(fig2, gmm_full, color='green')

grid = gridplot([[fig1, fig2]])
show(grid)